In [1]:
import pandas as pd
import numpy as np 

In [2]:
file_path = "C:\\Users\\tonny\\OneDrive\\Desktop\\DAKI2 - Repo\\AI & Data\\Miniprojekt\\medical_students_dataset.csv"

df = pd.read_csv(file_path)

if df is None:
    print("File not valid")

In [ ]:
''' 
DUPLICATES HANDLING
SLOW but MERGES duplicate row

'''
# Function to take the first non-null value in a Series
def first_non_null(series):
    non_null = series.dropna()
    if not non_null.empty:
        return non_null.iloc[0]
    return np.nan

# Group by 'Student ID' to get the size of each group
group_sizes = df.groupby('Student ID').size()

# Identify unique IDs (only one row) and duplicate IDs (more than one row)
unique_ids = group_sizes[group_sizes == 1].index
duplicate_ids = group_sizes[group_sizes > 1].index

# Split the DataFrame
unique_df = df[df['Student ID'].isin(unique_ids)]
duplicate_df = df[df['Student ID'].isin(duplicate_ids)]

# Merge duplicates: For each duplicate group, aggregate columns using first_non_null
merged_duplicates = duplicate_df.groupby('Student ID', as_index=False).agg(first_non_null)

# Combine the unique rows with the merged duplicate rows
result_df = pd.concat([unique_df, merged_duplicates], ignore_index=True)

# Sort by Student ID and remove empty rows
df = result_df.sort_values('Student ID').reset_index(drop=True)

In [5]:
'''
DUPLICATES HANDLING
FAST but ONLY KEEPS duplicate rows with LEAST NaN values

'''
# Count missing values per row
df['missing_count'] = df.isna().sum(axis=1)

# Keep the row with the fewest missing values for each Student ID
df = df.loc[df.groupby('Student ID')['missing_count'].idxmin()]

# Drop helper column
df = df.drop(columns='missing_count')

# Sort by Student ID and remove empty rows
df = df.sort_values('Student ID').reset_index(drop=True)

In [7]:
# Use interpolation to add missing Student ID's
df['Student ID'] = df['Student ID'].interpolate(limit_direction='both').astype(float)

In [9]:
# Identify duplicate student_id rows
duplicates = df[df['Student ID'].duplicated()]

# Only proceed if we actually have duplicates
if not duplicates.empty:
    # Count how many times each Student ID occurs
    id_counts = df['Student ID'].value_counts()

    # Only keep counts > 1 (actual duplicates)
    duplicates = id_counts[id_counts > 1]

    # Print the Student IDs which have duplicates
    print("Duplicate counts:")
    print(duplicates)
else:
    print("No duplicates found!")

No duplicates found!


In [10]:
print(df.head)

<bound method NDFrame.head of        Student ID   Age  Gender      Height     Weight Blood Type        BMI  \
0             1.0  18.0  Female  161.777924  72.354947          O  27.645835   
1             2.0  34.0    Male  152.069157  47.630941          B  20.597139   
2             3.0  32.0  Female  182.537664  55.741083          A  16.729017   
3             4.0  30.0    Male  182.112867  63.332207          B  19.096042   
4             5.0  23.0  Female  179.339293  46.234173          O  14.375143   
...           ...   ...     ...         ...        ...        ...        ...   
98971     99995.0  22.0    Male         NaN  70.282622        NaN  27.631082   
98972     99997.0  29.0  Female  163.917675  45.225194        NaN  16.831734   
98973     99998.0  34.0     NaN  173.275523  99.648914          B  33.189303   
98974     99999.0  30.0  Female  156.446944  50.142824          A  20.486823   
98975    100000.0  20.0  Female  153.927409  99.928405        NaN  42.175189   

       Te